<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20Deployment/Course%203%20-%20TensorFlow%20Datasets/Week%201/Examples/horse-or-human.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Good to run this to ensure you are using TF2.x

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

data = tfds.load('horses_or_humans', split='train', as_supervised=True)
val_data = tfds.load('horses_or_humans', split='test', as_supervised=True)

train_batches = data.shuffle(100).batch(32)
validation_batches = val_data.batch(32)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', 
                                      input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_batches, epochs=10, validation_data=validation_batches, validation_steps=1)

Epoch 1/10
33/33 [==============================] - 5s 163ms/step - loss: 7.0531 - accuracy: 0.7128 - val_loss: 0.4535 - val_accuracy: 0.8438
Epoch 2/10
33/33 [==============================] - 4s 123ms/step - loss: 0.1301 - accuracy: 0.9455 - val_loss: 1.6637 - val_accuracy: 0.7500
Epoch 3/10
33/33 [==============================] - 4s 124ms/step - loss: 0.2554 - accuracy: 0.8870 - val_loss: 0.3239 - val_accuracy: 0.9688
Epoch 4/10
33/33 [==============================] - 4s 123ms/step - loss: 0.0697 - accuracy: 0.9747 - val_loss: 0.2275 - val_accuracy: 0.9375
Epoch 5/10
33/33 [==============================] - 4s 123ms/step - loss: 0.0344 - accuracy: 0.9873 - val_loss: 0.6786 - val_accuracy: 0.9062
Epoch 6/10
33/33 [==============================] - 4s 122ms/step - loss: 0.0064 - accuracy: 0.9990 - val_loss: 0.7031 - val_accuracy: 0.9062
Epoch 7/10
33/33 [==============================] - 4s 123ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 1.5573 - val_accuracy: 0.8750
Epoch 